In [3]:
import torch
from matplotlib import pyplot as plt
import cv2
from PIL import Image
from pathlib import Path
import numpy as np


In [4]:

YOLOV5_PATH = Path("/Users/taaha/codes/pytorch/ultralytics_yolov5/")
import sys
sys.path.append(str(YOLOV5_PATH))
import detect as dt
from utils.dataloaders import LoadWebcam
from utils.plots import Colors
weights = YOLOV5_PATH / 'runs' / 'train' / 'amin_ds' / 'weights' / 'best.pt'
data = YOLOV5_PATH / 'datasets' / 'amin_ds' / 'amin_ds.yaml'


In [ ]:
import os
# os.chdir('/Users/taaha/codes/ros/project_ws/src/ros_serial_test/scripts/')
os.getcwd()

In [5]:

#Model
model = torch.hub.load(str(YOLOV5_PATH), 'custom', path=str(weights), source='local') # local repo
# Images
img = cv2.imread('./table.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY )
# Inference
results = model(img, size=640)  # includes NMS

# Results
results.print()
# results.show()  # or .show()

#results = results.xyxy[0]  # img1 predictions (tensor)
boxes = results.pandas().xywh[0]  # img1 predictions (pandas)
print(boxes)


YOLOv5 🚀 v6.1-395-g4d05472 Python-3.8.13 torch-1.12.1 CPU

Fusing layers... 
[W NNPACK.cpp:51] Could not initialize NNPACK! Reason: Unsupported hardware.
Model summary: 444 layers, 86200330 parameters, 0 gradients, 203.8 GFLOPs
Adding AutoShape... 


image 1/1: 960x1280 1 paper, 2 plastic_bottles
Speed: 20.6ms pre-process, 1486.9ms inference, 1.2ms NMS per image at shape (1, 3, 480, 640)
       xcenter     ycenter       width      height  confidence  class  \
0  1002.861267  688.384277  149.987915  260.274902    0.908981      2   
1   510.019592  500.138184  128.642242  145.350952    0.292232      1   
2    82.097855  714.043335  164.195709  485.542480    0.287481      2   

             name  
0  plastic_bottle  
1           paper  
2  plastic_bottle  


In [ ]:
print(results.pandas().xywh[0])

In [ ]:
import pandas as pd
print('hhhhhhh')
print(results.pandas().xywh[0].itertuples())


# respd.reset_index()
for idx, (xc, yc, w, h, conf, cls, name) in results.pandas().xywh[0].iterrows():
    print(f"xcenter:{xc}, ycenter:{yc}, width:{w}, height:{h}, confidence:{conf*100:.2f}%, class:{cls}, name:{name}")


In [35]:
im = cv2.imread('./table.jpg')
xc, yc = 900, 100

im = cv2.circle(img=im, center=(xc,yc), radius=10,color=(0,0,255), thickness=-1)

im_h, im_w, channel = im.shape
name = 'Hello'
text_size = cv2.getTextSize(name, cv2.FONT_HERSHEY_SIMPLEX, fontScale=(1.1e-3 * im.shape[0]), thickness=2)
tx_width, tx_height = text_size[0][0], text_size[0][1]
pt1 = (((xc+5) if (xc+10+tx_width < im_w) else (xc-tx_width-5)), ((yc-5) if (yc-10-tx_height > 0) else (yc+5+tx_height))) # bottom-left
pt2 = (pt1[0]+10+tx_width, pt1[1]-10-tx_height) # top-right
pt3 = (pt1[0], pt1[1]-10-tx_height) # top-left
pt4 = (pt1[0]+10+tx_width, pt1[1]) # bottom-right
# region = np.array([list(pt1),list(pt3),list(pt2),list(pt4)], dtype='int32')
region = np.array([pt1,pt3,pt2,pt4], dtype='int32')
print(pt1,pt2)
print(im.shape)

im = cv2.rectangle(img=im, pt1=pt1, pt2=pt2, color=(0,0,255))
im = cv2.fillPoly(img=im, pts=[region], color=(0,0,255))
im = cv2.putText(img=im, text=name, org=(pt1[0]+5, pt1[1]-5), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=(1.1e-3 * im.shape[0]), color=(255,255,255), thickness=2)

cv2.imwrite('image.png', im)

(905, 95) (996, 62)
(960, 1280, 3)


True

In [39]:
colors = Colors()
def add_dot_and_label(img, xc, yc, conf, cls, name, clr=None):
    if not clr: clr = colors(int(cls))
    label = f"{name}:{conf*100:.2f}%"

    im_h, im_w, channel = img.shape
    text_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, fontScale=(1.1e-3 * im_h), thickness=2)
    tx_width, tx_height = text_size[0][0], text_size[0][1]

    pt1 = (((xc+5) if (xc+10+tx_width < im_w) else (xc-tx_width-5)), ((yc-5) if (yc-10-tx_height > 0) else (yc+5+tx_height))) # bottom-left
    pt2 = (pt1[0]+10+tx_width, pt1[1]-10-tx_height) # top-right
    pt3 = (pt1[0], pt1[1]-10-tx_height) # top-left
    pt4 = (pt1[0]+10+tx_width, pt1[1]) # bottom-right
    # region = np.array([list(pt1),list(pt3),list(pt2),list(pt4)], dtype='int32')
    region = np.array([pt1,pt3,pt2,pt4], dtype='int32')    

    img = cv2.circle(img=img, center=(xc,yc), radius=10,color=clr, thickness=-1)
    img = cv2.rectangle(img=img, pt1=pt1, pt2=pt2, color=clr)
    img = cv2.fillPoly(img=img, pts=[region], color=clr)
    img = cv2.putText(img=img, text=label, org=(pt1[0]+5, pt1[1]-5), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=(1.1e-3 * im_h), color=(255,255,255), thickness=2)
    return img

# Webcam

In [7]:
cap = cv2.VideoCapture(0)

batch_size = 1
images = []

while True:
    ret, img = cap.read()
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    pred = model(img)
    print(f"type of image{type(img)}, dim:{img.shape}")

    mps_img = torch.from_numpy(img).to(torch.device('mps'))
    print(f"type of tensor{type(mps_img)}, dim:{mps_img.shape}")
    for idx, (xc, yc, w, h, conf, cls, name) in pred.pandas().xywh[0].iterrows():
        img = add_dot_and_label(img, int(xc), int(yc), conf, cls, name)
        cv2.imshow('video with bboxes', img)
    
    if cv2.waitKey(1) == 27: 
        break  # esc to quit
    cv2.destroyAllWindows()

type of image<class 'numpy.ndarray'>, dim:(1080, 1920, 3)
type of tensor<class 'torch.Tensor'>, dim:torch.Size([1080, 1920, 3])
type of image<class 'numpy.ndarray'>, dim:(1080, 1920, 3)
type of tensor<class 'torch.Tensor'>, dim:torch.Size([1080, 1920, 3])
type of image<class 'numpy.ndarray'>, dim:(1080, 1920, 3)
type of tensor<class 'torch.Tensor'>, dim:torch.Size([1080, 1920, 3])
type of image<class 'numpy.ndarray'>, dim:(1080, 1920, 3)
type of tensor<class 'torch.Tensor'>, dim:torch.Size([1080, 1920, 3])
type of image<class 'numpy.ndarray'>, dim:(1080, 1920, 3)
type of tensor<class 'torch.Tensor'>, dim:torch.Size([1080, 1920, 3])
type of image<class 'numpy.ndarray'>, dim:(1080, 1920, 3)
type of tensor<class 'torch.Tensor'>, dim:torch.Size([1080, 1920, 3])
type of image<class 'numpy.ndarray'>, dim:(1080, 1920, 3)
type of tensor<class 'torch.Tensor'>, dim:torch.Size([1080, 1920, 3])
type of image<class 'numpy.ndarray'>, dim:(1080, 1920, 3)
type of tensor<class 'torch.Tensor'>, dim:torc

KeyboardInterrupt: 

In [ ]:
device = torch.device('mps')

model.to(device)
stride, names, pt = model.stride, model.names, model.pt
imgsz = dt.check_img_size((640, 640), s=stride)  # check image size

dataset = LoadWebcam(pipe=0)
bs = len(dataset)

model.warmup(imgsz=(1 if pt else bs, 3, *imgsz))  # warmup
seen, windows, dt = 0, [], [0.0, 0.0, 0.0]
for path, im, im0s, vid_cap, s in dataset:
    t1 = dt.time_sync()
    im = torch.from_numpy(im).to(device)
    im = im.half() if model.fp16 else im.float()  # uint8 to fp16/32
    im /= 255  # 0 - 255 to 0.0 - 1.0
    if len(im.shape) == 3:
        im = im[None]  # expand for batch dim
    t2 = dt.time_sync()
    dt[0] += t2 - t1
    pred = model(im)
    t3 = dt.time_sync()
    dt[1] += t3 - t2
    pred = dt.non_max_suppression(pred, conf_thres, iou_thres, classes, agnostic_nms, max_det=max_det)
    dt[2] += dt.time_sync() - t3

    for i, det in enumerate(pred):

In [9]:
import math
def inv_kinematics(r, z, l1=20, l2=22):
        if not (-1 <= (r**2+z**2-l1**2-l2**2)/(2*l1*l2) <= 1):
            # domain test of acos
            # also limit that is possible for the two links to reach
            return

        # Equations for Inverse kinematics
        phi_2 = (math.acos((r**2+z**2-l1**2-l2**2)/(2*l1*l2)))  # eqn 2
        phi_1 = math.atan2(z, r) + math.atan2(l2*math.sin(phi_2), (l1 + l2*math.cos(phi_2))) # eqn 3
        phi_2 = phi_2
        #theta_1 = rad2deg(phi_2-phi_1)  # eqn 4 converted to degrees

        #phi_3 = arccos((r1**2-arm1**2-arm1**2)/(-2*arm1*arm2))
        #theta_2 = 180-rad2deg(phi_3)

        theta1 = math.degrees(phi_1)
        theta2 = math.degrees(phi_2)

        #print('theta 1: ', phi_1)
        #print('theta two: ', phi_2)

        return theta1, theta2


In [11]:
inv_kinematics(r=12, z=14)

(119.0893533346236, 128.18348243320992)